# Model predictive control of a bilinear motor


Source paper: "Linear predictors for nonlinear dynamical systems: Koopman operator meets model predictive control" by Milan Korda and Igor Mezic https://arxiv.org/pdf/1611.03537.pdf 


Source code: https://github.com/MilanKorda/KoopmanMPC/

In this tutorial, we will lift nonlinear dynamics of a model representing a motor engine into a higher dimensional space where it's evolution is approximately linear. The Koopman operator is extended to controlled dynamical systems. To compute a finite-dimensional approximation of the operator, we use Extended Dynamic Mode Decomposition (EDMD). Essentially, this ledas to a linear controlled dynamical system, which we use in for model predictive control (MPC). 

$$x_1 = −(R_a/L_a)x_1 − (k_m/L_a)x_2 u + u a/L_a \\ 
\dot{x}_2 = −(B/J)x_2 + (k_m/J)x_1 u − \tau l / J \\
y = x_2$$

The goal of the control is to follow the angular velocity reference, represented by the variable $y_r$, which is reflected in the objective function minimized in the Model Predictive Control (MPC) problem:

$$J = (Cz_{N_p} - y_r)^T Q_{N_p} (Cz_{N_p} - y_r) + \sum_{i=0}^{N_p-1} (Cz_{i} - y_r)^T Q (Cz_{i} - y_r) + u_i^T R u_i$$

where $C=[1,0,\ldots, 0]$ projects the intrinsic state $z$ to its first coordinate. The cost function matrices were selected as $Q = Q_{N_p} = 1$ and $R = 0.01$. The prediction horizon is set to one second, resulting in $N_p = 100$.

where $x_1$ is the rotor current, $x_2$ the angular velocity and the control input $u$ is the stator
current and the output $y$ is the angular velocity. The parameters are $L_a = 0.314$, $Ra = 12.345$, $k_m = 0.253$, $J = 0.00441$, $B = 0.00732$, $\tau l = 1.47$, $u_a = 60$

The objective is to create a Model Predictive Control (MPC) controller using only the input-output data of a motor model, without any prior knowledge of the model's structure. To achieve this, we will begin by discretizing the dynamic behavior of the motor using the Runge-Kutta four method with a time step of 0.01 seconds. We will then simulate 200 different scenarios over the course of 1000 sampling periods, using a randomly generated signal as the control input.

The lifting functions used are the time-delayed vector and 100 thin plate spline radial basis functions (with uniformly distributed centers over $[-1,1]^3$. Together with the original states this results in a lifted state-space dimension of $N = 103$. 

To evaluate the accuracy of our data-driven model, we will plot results of two initial conditions and compare it to the true motor model. This will enable us to assess the predictive performance of our Koopman operator-based model before integrating it into the Model Predictive Control (MPC) framework.

The original paper also compares the model against a predictor based on local linarization (cf. Table 3). The Koopman operator-based predictor demonstrates superior accuracy, particularly when for predictions over a longer time horizon.

We will be evaluating the performance of the  Koopman operator-based MPC controller in two different scenarios. In the first scenario, there are no constraints imposed on the output and the objective is to track a piecewise constant reference. In the second scenario, we will impose the constraint of $y ∈ [−0.4, 0.4]$ and track a time-varying reference of $y_r(t) = 0.5\cos(2\pi t/3)$, which may violate the output constraint during some part of the simulation period. The original paper shows that the controller based on local linearization fails for the second case.